# Синхронизация с гит

In [ ]:
# --- БЛОК №1: НАЧАЛО РАБОТЫ (выполнять в каждой новой сессии) ---

from google.colab import drive, userdata
import os

# 1. Монтируем Google Drive
drive.mount('/content/drive')

# 2. Получаем секретный токен
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

# 3. Указываем ваше имя и email (как на GitHub)
GIT_USERNAME = "kirichich1" # <-- Ваше имя пользователя
GIT_EMAIL = "kirichich@bk.ru" # <-- Ваша почта

# 4. !!! ПРЕДСТАВЛЯЕМСЯ СИСТЕМЕ GIT !!!
# Эта конфигурация будет действовать до конца текущей сессии
!git config --global user.name "{GIT_USERNAME}"
!git config --global user.email "{GIT_EMAIL}"

# 5. Переходим в директорию проекта
PROJECT_PATH = "/content/drive/MyDrive/GitHub/no2_prediction_pipeline"
%cd {PROJECT_PATH}

# 6. (Рекомендуется) Скачиваем последние изменения с GitHub на случай, если вы работали с другого устройства
!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GitHub/no2_prediction_pipeline
Already up to date.


# Импорт нужных инструментов


In [16]:
import ee
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
ee.Authenticate()
ee.Initialize(project='pollutionmlproject')

In [18]:
!pip install earthengine-api

# Выгрузка данных

In [19]:
# --- ПАРАМЕТРЫ ГОРОДА ---
CITY_GEOM = ee.Geometry.Point(73.5, 61.25).buffer(25000)
CITY_NAME = 'Surgut'

# ==============================================================================
# == УСТАНОВИТЕ ГОД ДЛЯ ВЫГРУЗКИ ===============================================
# ==============================================================================
YEAR_TO_DOWNLOAD = 2023 # <--- МЕНЯЙТЕ ГОД ЗДЕСЬ
# ==============================================================================

# --- Параметры фильтрации S5P ---
PIXEL_CLOUD_THRESHOLD = 0.5
SOLAR_ZENITH_ANGLE_THRESHOLD = 80.0

# ==============================================================================
# == БЛОК 2: ФУНКЦИИ-СБОРЩИКИ ДАННЫХ (S5P обновлена для AAI) ===================
# ==============================================================================

def get_s5p_data_for_date(target_date, geometry):
    """
    Получает данные NO2 и AAI за одну дату.
    КРИТИЧЕСКИ ВАЖНО: Маска облачности берется из NO2 и применяется к AAI.
    """
    try:
        start = ee.Date(target_date)
        end = start.advance(1, 'day')

        # 1. Запрашиваем коллекцию NO2 (источник маски облачности)
        s5p_no2_coll = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2') \
            .filterBounds(geometry).filterDate(start, end)

        # 2. Запрашиваем коллекцию AAI (данные, которые нужно отфильтровать)
        s5p_aai_coll = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_AER_AI') \
            .filterBounds(geometry).filterDate(start, end)

        # Если нет снимка NO2, мы не можем ни получить NO2, ни сделать маску для AAI.
        if s5p_no2_coll.size().getInfo() == 0:
            return None

        # Берем средние снимки за день
        image_no2 = s5p_no2_coll.mean()

        # Проверяем, есть ли AAI. Если нет, создаем пустую переменную, чтобы код не упал
        has_aai = s5p_aai_coll.size().getInfo() > 0
        image_aai = s5p_aai_coll.mean() if has_aai else ee.Image.constant(None)

        # --- СОЗДАНИЕ МАСКИ (по данным NO2) ---
        cloud_mask = image_no2.select('cloud_fraction').lt(PIXEL_CLOUD_THRESHOLD)
        sza_mask = image_no2.select('solar_zenith_angle').lt(SOLAR_ZENITH_ANGLE_THRESHOLD)
        combined_mask = cloud_mask.And(sza_mask)

        # --- ПРИМЕНЕНИЕ МАСКИ К ОБОИМ ПРОДУКТАМ ---
        # Фильтруем NO2
        no2_filtered = image_no2.updateMask(combined_mask)

        # Фильтруем AAI той же самой маской! (Реализация вашего плана)
        aai_filtered = image_aai.updateMask(combined_mask) if has_aai else image_aai

        # Сбор статистики (Reduce Region)
        stats_no2 = no2_filtered.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=1000,
            maxPixels=1e9
        ).getInfo()

        # Отдельно собираем облачность (без маски облачности, чтобы знать реальное покрытие)
        cloud_stats = image_no2.select('cloud_fraction').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=1000,
            maxPixels=1e9
        ).getInfo()

        stats_aai = {}
        if has_aai:
            stats_aai = aai_filtered.select('absorbing_aerosol_index').reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e9
            ).getInfo()

        return {
            'no2_trop_mean': stats_no2.get('tropospheric_NO2_column_number_density'),
            'cloud_frac_mean': cloud_stats.get('cloud_fraction'),
            'aai_mean': stats_aai.get('absorbing_aerosol_index') if has_aai else None
        }

    except Exception as e:
        print(f"Ошибка S5P на дате {target_date}: {e}")
        return None

def get_era5_data_for_date(target_date, geometry):
    """
    Функция ERA5 без изменений.
    """
    try:
        start = ee.Date(target_date)
        end = start.advance(1, 'day')
        era5_bands = ['temperature_2m', 'surface_pressure',
                      'u_component_of_wind_10m', 'v_component_of_wind_10m']

        image = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
                  .filterBounds(geometry) \
                  .filterDate(start, end) \
                  .select(era5_bands) \
                  .mean()

        if image:
            data = image.reduceRegion(
                reducer=ee.Reducer.mean(), geometry=geometry, scale=11132, maxPixels=1e9
            ).getInfo()

            temp_celsius = data.get('temperature_2m') - 273.15 if data.get('temperature_2m') else None
            pressure_hpa = data.get('surface_pressure') / 100 if data.get('surface_pressure') else None

            return {
                'temperature_celsius': temp_celsius,
                'pressure_hpa': pressure_hpa,
                'u_wind_10m': data.get('u_component_of_wind_10m'),
                'v_wind_10m': data.get('v_component_of_wind_10m')
            }
        return None
    except Exception as e:
        print(f"Ошибка ERA5 на дате {target_date}: {e}"); return None

# ==============================================================================
# == БЛОК 3: ОСНОВНОЙ ЦИКЛ СБОРА ДАННЫХ =========================================
# ==============================================================================

all_data = []
print(f"Начинаю сбор данных (NO2 + AAI + ERA5) для города: {CITY_NAME} за {YEAR_TO_DOWNLOAD} год")

start_of_period = f"{YEAR_TO_DOWNLOAD}-02-01"
end_of_period = f"{YEAR_TO_DOWNLOAD}-09-30"
print(f"--- Период: с {start_of_period} по {end_of_period} ---")
date_range = pd.date_range(start=start_of_period, end=end_of_period, freq='D')

for date in tqdm(date_range, desc=f"Год {YEAR_TO_DOWNLOAD}"):
    current_date_str = date.strftime('%Y-%m-%d')

    # Теперь эта функция возвращает и NO2, и AAI
    s5p_data = get_s5p_data_for_date(current_date_str, CITY_GEOM)
    era5_data = get_era5_data_for_date(current_date_str, CITY_GEOM)

    row = {'date': date, 'city': CITY_NAME}

    if s5p_data:
        row.update(s5p_data)
    if era5_data:
        row.update(era5_data)

    all_data.append(row)

# ==============================================================================
# == БЛОК 4: ФОРМИРОВАНИЕ И СОХРАНЕНИЕ ИТОГОВОГО ФАЙЛА ========================
# ==============================================================================
final_df = pd.DataFrame(all_data)

# Обновленный порядок колонок, включая AAI
column_order = [
    'date', 'city',
    'no2_trop_mean', 'aai_mean', 'cloud_frac_mean', # Добавил aai_mean сюда
    'temperature_celsius', 'pressure_hpa', 'u_wind_10m', 'v_wind_10m'
]

# Пересортировка колонок (с защитой, если вдруг данных не было вообще и колонка не создалась)
existing_cols = [col for col in column_order if col in final_df.columns]
final_df = final_df[existing_cols]

# Заполнение пропусков AAI нулями (согласно вашему Плану Шаг 2)
if 'aai_mean' in final_df.columns:
    final_df['aai_mean'] = final_df['aai_mean'].fillna(0)
else:
    # Если за весь период не нашлось ни одного пикселя AAI, создаем колонку с нулями
    final_df['aai_mean'] = 0

drive.mount('/content/drive')

output_filename = f'{CITY_NAME}_data_{YEAR_TO_DOWNLOAD}_Feb-Sep_WITH_AAI.csv'
final_df.to_csv(f'/content/drive/MyDrive/GitHub/no2_prediction_pipeline/data/raw/add_predictors/{output_filename}', index=False)

print(f"\nСбор данных завершен.")
print(f"Данные по AAI отфильтрованы по маске облачности NO2 (< {PIXEL_CLOUD_THRESHOLD}).")
print(f"Пропуски в AAI заполнены нулями.")
print(f"Файл сохранен: {output_filename}")

Начинаю сбор данных (NO2 + AAI + ERA5) для города: Surgut за 2019 год
--- Период: с 2019-10-01 по 2019-11-01 ---


Год 2019:   0%|          | 0/32 [00:00<?, ?it/s]

NameError: name 'drive' is not defined

In [20]:
final_df.head(50)

,date,city,no2_trop_mean,aai_mean,cloud_frac_mean,temperature_celsius,pressure_hpa,u_wind_10m,v_wind_10m
0,2019-10-01,Surgut,0.000016,-1.243787,0.316108,5.470216,1008.068635,3.066199,3.149412
1,2019-10-02,Surgut,NaN,0.000000,0.738776,8.652330,1006.436287,3.301975,3.697150
2,2019-10-03,Surgut,NaN,0.000000,0.715656,8.576442,1003.105599,2.583664,3.565559
3,2019-10-04,Surgut,0.000016,-1.105521,0.091854,5.378248,1007.605599,4.119870,0.419915
4,2019-10-05,Surgut,0.000018,-1.334434,0.257991,6.840592,1009.571978,0.387140,2.072185
5,2019-10-06,Surgut,0.000044,-1.118953,0.456951,10.251430,1008.463693,2.879484,1.608436
6,2019-10-07,Surgut,NaN,0.000000,0.858811,5.586874,1016.612212,-0.869909,-0.321341
7,2019-10-08,Surgut,0.000024,-1.063489,0.092424,5.396470,1019.306304,-2.759855,1.200947
8,2019-10-09,Surgut,0.000021,-1.150971,0.074048,7.811259,1013.992606,-2.727086,2.947971
9,2019-10-10,Surgut,NaN,-0.710787,0.367239,9.731291,1009.804224,-1.667444,3.501615


In [ ]:
final_df['no2_trop_mean'].isna().sum()

np.int64(76)

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 242 non-null    datetime64[ns]
 1   city                 242 non-null    object        
 2   no2_trop_mean        166 non-null    float64       
 3   aai_mean             242 non-null    float64       
 4   cloud_frac_mean      242 non-null    float64       
 5   temperature_celsius  242 non-null    float64       
 6   pressure_hpa         242 non-null    float64       
 7   u_wind_10m           242 non-null    float64       
 8   v_wind_10m           242 non-null    float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 17.1+ KB


In [ ]:
final_df['no2_trop_mean'].isna().sum()

np.int64(76)

# Коммит для гит

In [ ]:
# --- БЛОК №2: СОХРАНЕНИЕ РЕЗУЛЬТАТОВ (в конце работы) ---

# 1. Проверяем статус (опционально)
!git status

# 2. ДОБАВЛЯЕМ файлы в "коробку" для отправки. Это то, что вы пропустили.
!git add .

# 3. ПОДПИСЫВАЕМ "коробку" (делаем коммит)
!git commit -m "add changes to data-export script"

# 4. ОТПРАВЛЯЕМ "коробку" на GitHub
!git push

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   notebooks/data_export_Surgut_with_AAI.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Surgut_data_2020_Feb-Sep_WITH_AAI.csv
	data/raw/add_predictors/Surgut_data_2020_Feb-Sep_WITH_AAI.csv

no changes added to commit (use "git add" and/or "git commit -a")
[main 34bfd33] add changes to data-export script
 3 files changed, 489 insertions(+), 1 deletion(-)
 create mode 100644 Surgut_data_2020_Feb-Sep_WITH_AAI.csv
 create mode 100644 data/raw/add_predictors/Surgut_data_2020_Feb-Sep_WITH_AAI.csv
 rewrite notebooks/data_export_Surgut_with_AAI.ipynb (98%)
Enumerating objects: 14, done.
Counting objects: 100% (14/14), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 1